In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

amazon_df = pd.read_csv('amazon_baby.csv')
amazon_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


# Question 1
Out of the 11 words in selected_words, which one is most used in the reviews in the dataset?

In [2]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [3]:
review_df = amazon_df[['review', 'rating']]
for word in selected_words:
    review_df[word] = review_df['review'].str.count(word)

word_dict = {}
for word in selected_words:
    word_dict[word] = review_df[word].sum()

word_dict

{'amazing': 2533.0,
 'awesome': 3380.0,
 'awful': 694.0,
 'bad': 5099.0,
 'fantastic': 1611.0,
 'great': 51091.0,
 'hate': 3795.0,
 'horrible': 1081.0,
 'love': 65236.0,
 'terrible': 1126.0,
 'wow': 135.0}

In [4]:
import operator

word_dict_sorted = sorted(word_dict.items(),key=operator.itemgetter(1),reverse=True)
word_dict_sorted[0]

('love', 65236.0)

# Question 2

Out of the 11 words in selected_words, which one is least used in the reviews in the dataset? 

In [5]:
word_dict_sorted[-1]

('wow', 135.0)

# Question 3
Out of the 11 words in selected_words, which one got the most positive weight in the selected_words_model?
(Tip: when printing the list of coefficients, make sure to use print_rows(rows=12) to print ALL coefficients.)

In [6]:
def cleanNaN(value):
    if pd.isnull(value):
        return ""
    else:
        return value
    
review_df['review'] = review_df['review'].apply(cleanNaN)
review_df['review'][30:50]

30    Beautiful little book.  A great little short s...
31    This book is so worth the money. It says 9+ mo...
32    we just got this book for our one-year-old and...
33    The book is colorful and is perfect for 6month...
34    The book is cute, and we are huge fans of Lama...
35    What a great book for babies!  I\'d been looki...
36    My son loved this book as an infant.  It was p...
37    Our baby loves this book & has loved it for a ...
38                                                     
39    My son likes brushing elmo\'s teeth. Almost to...
40    This was a birthday present for my 2 year old ...
41    This bear is absolutely adorable and I would g...
42    My baby absolutely loves Elmo and so this book...
43    I bought two for recent baby showers!  The boo...
44    We wanted to get another book like the Big Bir...
45    This is a cute little peek-a-boo story book.  ...
46    My 3 month old son loves this book. We read it...
47    Very cute interactive book! My son loves t

In [9]:
#ignore all 3* reviews
review_df = review_df[review_df['rating'] != 3]

review_df['sentiment'] = review_df['rating'] >= 4
review_df.head()

,review,rating,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate,sentiment
1,it came early and was not disappointed. i love...,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,Very soft and comfortable and warmer than it l...,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,This is a product well worth the purchase. I ...,5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,All of my kids have cried non-stop when I trie...,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
5,"When the Binky Fairy came to our house, we did...",5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train, y_test = train_test_split(review_df['review'], review_df['sentiment'], test_size=0.2, random_state=42)

vect = CountVectorizer(vocabulary=selected_words)

vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_dtm, y_train)


X_test_dtm = vect.transform(X_test)

# class predictions and predicted probabilities
y_pred_class = log_reg_model.predict(X_test_dtm)
y_pred_prob = log_reg_model.predict_proba(X_test_dtm)[:, 1]

# calculate accuracy and AUC
from sklearn import metrics
print("Accuracy score:",metrics.accuracy_score(y_test, y_pred_class))
print("AUC:",metrics.roc_auc_score(y_test, y_pred_prob))

Accuracy score: 0.849000029984
AUC: 0.691302082365


In [13]:
selected_words_model.coef_

array([[  4.46737080e-01,  -5.39072607e-01,  -2.06144827e-02, ...,
          2.12102555e-06,  -1.47136002e-02,   1.58560621e-03],
       [  6.48066977e-02,   3.82531089e-01,  -1.77927006e-02, ...,
         -1.12316284e-05,  -3.53566768e-02,  -4.49031149e-01],
       [ -9.42010726e-02,  -6.25262789e-01,  -5.60486632e-02, ...,
         -2.13497659e-04,  -1.58853467e-02,  -1.13166445e-01],
       [ -2.09244157e-02,   5.37525144e-01,  -2.11252736e-01, ...,
          3.65222961e-01,  -1.11001532e-01,  -2.14355021e-03],
       [ -3.51197458e-01,  -3.94838183e-01,   1.09626717e-01, ...,
         -7.26879320e-01,   1.56471128e-01,  -2.83748653e-03]])

In [9]:
selected_words_model.intercept_

array([-2.07997071, -2.46404718, -2.09170841, -1.48075333,  0.09154976])

# Question 4
Out of the 11 words in selected_words, which one got the most negative weight in the selected_words_model?
(Tip: when printing the list of coefficients, make sure to use print_rows(rows=12) to print ALL coefficients.)

# Question 5
Which of the following ranges contains the accuracy of the selected_words_model on the test_data?

In [11]:
from sklearn.metrics import accuracy_score

y_pred = selected_words_model.predict(X_test)

accuracy_score(y_test, y_pred)

0.58313536603970972

# Question 6
Which of the following ranges contains the accuracy of the sentiment_model in the IPython Notebook from lecture on the test_data?

# Question 7
Which of the following ranges contains the accuracy of the majority class classifier, which simply predicts the majority class on the test_data?